# Data Import

In [1]:
import pandas as pd
import os
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from geopy.distance import geodesic

from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import silhouette_score

from sklearn.decomposition import PCA

from mpl_toolkits.mplot3d import Axes3D

import plotly.graph_objs as go

In [2]:
dataset_path = '/kaggle/input/sncb-data-augumentation/enriched_cleaned_ar41_for_ulb.csv'

# Check if the file exists before trying to read it
if os.path.exists(dataset_path):
    data = pd.read_csv(dataset_path)

    # Display the basic information and the first few rows of the dataframe
    data_info = data.info()
    data_head = data.head()

    # If you want to print the information to the console
    print(data_info)
    print(data_head)
else:
    print(f"The file {dataset_path} does not exist.")
    
data = data.drop(['Unnamed: 0', 'dayofweek', 'datetime', 'date_hour'], axis=1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17677337 entries, 0 to 17677336
Data columns (total 32 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Unnamed: 0          int64  
 1   timestamps_UTC      object 
 2   mapped_veh_id       float64
 3   lat                 float64
 4   lon                 float64
 5   RS_E_InAirTemp_PC1  float64
 6   RS_E_InAirTemp_PC2  float64
 7   RS_E_OilPress_PC1   float64
 8   RS_E_OilPress_PC2   float64
 9   RS_E_RPM_PC1        float64
 10  RS_E_RPM_PC2        float64
 11  RS_E_WatTemp_PC1    float64
 12  RS_E_WatTemp_PC2    float64
 13  RS_T_OilTemp_PC1    float64
 14  RS_T_OilTemp_PC2    float64
 15  date                object 
 16  hour                float64
 17  dayofweek           float64
 18  weekday             object 
 19  Distance            float64
 20  Speed               float64
 21  date_hour           object 
 22  datetime            object 
 23  weather_main        object 
 24  temp                fl

# Label about Stoped or Running

In [3]:
# def grid_search_weights(data, feature_columns, weight_grid, n_clusters=2, random_state=0):
#     best_score = -1
#     best_weights = None
#     best_clusters = None
#     scaler = StandardScaler()

#     data_np = data.to_numpy()

#     for weights in weight_grid:
#         data_scaled = data_np * np.array(weights)
#         data_scaled = scaler.fit_transform(data_scaled)
#         kmeans = KMeans(n_clusters=n_clusters, random_state=random_state)
#         clusters = kmeans.fit_predict(data_scaled)
#         score = silhouette_score(data_scaled, clusters)
        
#         if score > best_score:
#             best_score = score
#             best_weights = weights
#             best_clusters = clusters 

#     return best_weights, best_clusters, best_score

In [4]:
# # Change the value in data[hout] to better clustering
# data['hour'] = data['hour'].replace(22, -2)
# data['hour'] = data['hour'].replace(23, -1)

# data['RPM_Mean'] = data[['RS_E_RPM_PC1', 'RS_E_RPM_PC2']].mean(axis=1)

# # Define a smaller weight grid for demonstration
# weight_grid = [
#     [1, 1, 1],
#     [2, 1, 1],
#     [1, 1, 2],
#     [1, 1, 1.5],
#     [1.5, 1, 1],
#     [1.5, 1, 1.5],
# ]

# # Call the grid search function
# best_weights, best_score = grid_search_weights(
#     data[['Speed', 'RPM_Mean', 'hour']],
#     ['Speed', 'RPM_Mean', 'hour'], 
#     weight_grid
# )

# print(f"Best weights: {best_weights}")
# print(f"Best silhouette score: {best_score}")

In [5]:
# for i, feature in enumerate(['Speed', 'RPM_Mean', 'hour']):
#     clustering_data_cleaned[feature] *= best_weights[i]

# scaler = StandardScaler()
# clustering_data_scaled = scaler.fit_transform(clustering_data_cleaned)

# kmeans = KMeans(n_clusters=2, random_state=0)
# clusters = kmeans.fit_predict(clustering_data_scaled)

# data_parsed = data.copy()
# data_parsed = data_parsed.loc[clustering_data_cleaned.index]  
# data_parsed['running'] = clusters

# data_parsed.head()

In [6]:
# # Change the value in data[hout] to better clustering
# data['hour'] = data['hour'].replace(22, -2)
# data['hour'] = data['hour'].replace(23, -1)

# data['RPM_Mean'] = data[['RS_E_RPM_PC1', 'RS_E_RPM_PC2']].mean(axis=1)

# # Define a smaller weight grid for demonstration
# weight_grid = [
#     [1, 1, 1],
#     [2, 1, 1],
#     [1, 1, 2],
#     [1, 1, 1.5],
#     [1.5, 1, 1],
#     [1.5, 1, 1.5],
# ]

# best_weights, best_clusters, best_score = grid_search_weights(
#     data[['Speed', 'RPM_Mean', 'hour']],
#     ['Speed', 'RPM_Mean', 'hour'], 
#     weight_grid
# )

# print(f"Best weights: {best_weights}")
# print(f"Best silhouette score: {best_score}")

# data['running'] = np.nan  
# data.loc[data.index, 'running'] = best_clusters

# data

In [7]:
# Change the value in data[hout] to better clustering
data['hour'] = data['hour'].replace(22, -2)
data['hour'] = data['hour'].replace(23, -1)

data['RPM_Mean'] = data[['RS_E_RPM_PC1', 'RS_E_RPM_PC2']].mean(axis=1)

# Selecting the relevant columns for clustering
clustering_data = data[['Speed', 'RPM_Mean', 'hour']]  # Make sure to use 'RS_E_RPM_PC2' for the second RPM

# Removing rows with NaN values
clustering_data_cleaned = clustering_data.dropna()

# Normalize the data
scaler = StandardScaler()
clustering_data_scaled = scaler.fit_transform(clustering_data_cleaned)

# Increase the weight of 'hour' and 'Speed' by multiplying them by a factor, let's say 2
weight_factor = 1
clustering_data_scaled[:, 0] *= 1.1 * weight_factor  # Speed
clustering_data_scaled[:, 2] *= 1.3 * weight_factor  # hour

# Applying K-Means Clustering
kmeans = KMeans(n_clusters=2, random_state=0)
clusters = kmeans.fit_predict(clustering_data_scaled)

# Adding the cluster labels to the original data for analysis
# Make sure 'data_parsed' is a copy of your original dataframe to add the clusters to
data_parsed = data.copy()
data_parsed = data_parsed.loc[clustering_data_cleaned.index]  # Align the indices
data_parsed['running'] = clusters

# Displaying the first few rows of the dataset with cluster labels
data_parsed.head()

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,timestamps_UTC,mapped_veh_id,lat,lon,RS_E_InAirTemp_PC1,RS_E_InAirTemp_PC2,RS_E_OilPress_PC1,RS_E_OilPress_PC2,RS_E_RPM_PC1,RS_E_RPM_PC2,...,temp,feels_like,pressure,humidity,wind,clouds,temp_celsius,feels_like_celsius,RPM_Mean,running
0,2023-01-23 07:25:08,102.0,51.02,3.77,17.0,18.0,210.0,210.0,858.0,839.0,...,274.54,271.75,1035.0,92.0,2.52,100.0,1.39,-1.4,848.5,1
1,2023-01-23 07:25:16,102.0,51.02,3.77,17.0,20.0,200.0,200.0,801.0,804.0,...,274.54,271.75,1035.0,92.0,2.52,100.0,1.39,-1.4,802.5,1
2,2023-01-23 07:25:37,102.0,51.02,3.77,19.0,20.0,193.0,207.0,803.0,808.0,...,274.54,271.75,1035.0,92.0,2.52,100.0,1.39,-1.4,805.5,1
3,2023-01-23 07:25:41,102.0,51.02,3.77,19.0,20.0,196.0,203.0,801.0,803.0,...,274.54,271.75,1035.0,92.0,2.52,100.0,1.39,-1.4,802.0,1
4,2023-01-23 07:26:10,102.0,51.02,3.77,19.0,21.0,200.0,203.0,795.0,807.0,...,274.54,271.75,1035.0,92.0,2.52,100.0,1.39,-1.4,801.0,1


# Visualize Runing Lable

In [8]:
# # Create a new dataset for 3D visualization with the original data
# data_3d = np.column_stack((data['Speed'].values, data['RPM_Mean'].values, data['hour'].values))

# # 3D Visualization
# fig = plt.figure(figsize=(18,15))
# ax = fig.add_subplot(111, projection='3d')
# ax.scatter(data_3d[:, 0], data_3d[:, 1], data_3d[:, 2], c=clusters, marker='o')

# ax.set_xlabel('Speed')
# ax.set_ylabel('Average RPM')
# ax.set_zlabel('Hour')

# plt.show()

In [9]:
# Create a new dataset for 3D visualization with the original data
data_3d = np.column_stack((data_parsed['Speed'].values, data_parsed['RPM_Mean'].values, data_parsed['hour'].values))

# Sample a subset of data for visualization to reduce memory usage
sample_size = 10000  # Adjust this based on your dataset size
indices = np.random.choice(data_3d.shape[0], sample_size, replace=False)
data_sample = data_3d[indices]
clusters_sample = clusters[indices]

trace = go.Scatter3d(
    x=data_sample[:, 0], 
    y=data_sample[:, 1], 
    z=data_sample[:, 2],
    mode='markers',
    marker=dict(
        size=5,
        color=clusters_sample,         # set color to an array/list of desired values
        colorscale='Viridis',          # choose a colorscale
        opacity=0.8
    )
)

data_plot = [trace]

layout = go.Layout(
    margin=dict(l=0, r=0, b=0, t=0),
    scene=dict(
        xaxis=dict(title='Speed'),
        yaxis=dict(title='Average RPM'),
        zaxis=dict(title='Hour')
    )
)

fig = go.Figure(data=data_plot, layout=layout)
fig.show()

In [10]:
# # calculate cluster number
# cluster_counts = data_parsed['running'].value_counts()
# print(cluster_counts)

# # calculate the statistics of cluster
# cluster_stats = data_parsed.groupby('running').describe()

# # group the data
# speed_stats = data_parsed.groupby('running')['Speed'].describe()
# rpm_stats = data_parsed.groupby('running')['RS_E_RPM_PC1'].describe()
# hour_stats = data_parsed.groupby('running')['hour'].describe()

# # print the calculating result
# print("\nOverall Cluster Stats:\n", cluster_stats)
# print("\nSpeed Stats by Cluster:\n", speed_stats)
# print("\nRPM Stats by Cluster:\n", rpm_stats)
# print("\nHour Stats by Cluster:\n", hour_stats)

# #  .agg() function is available here
# detailed_stats = data_parsed.groupby('running').agg(['mean', 'std', 'min', 'max', 'median', 'count'])

# print("\nDetailed Stats by Cluster:\n", detailed_stats)

# Statistical methods to Label

In [11]:
# # Calculate Z-score for numerical features
# numerical_cols = data.select_dtypes(include=['float64', 'int64']).columns
# data_zscore = data[numerical_cols].apply(lambda x: (x - x.mean()) / x.std())

# # Define a threshold for Z-score
# zscore_threshold = 3

# # Flag data points where the absolute Z-score is greater than the threshold
# data['Outlier_ZScore'] = (data_zscore.abs() > zscore_threshold).any(axis=1)

# # Calculate IQR for numerical features
# Q1 = data[numerical_cols].quantile(0.25)
# Q3 = data[numerical_cols].quantile(0.75)
# IQR = Q3 - Q1

# # Define a threshold for IQR
# iqr_threshold = 1.5

# # Flag data points where the feature is outside the range of IQR
# data['Outlier_IQR'] = ((data[numerical_cols] < (Q1 - iqr_threshold * IQR)) | 
#                         (data[numerical_cols] > (Q3 + iqr_threshold * IQR))).any(axis=1)

# # Display the head of the dataframe to verify the new columns
# data.head()

In [12]:
# # Define a function to detect outliers within each segment
# def detect_outliers(df, segment):
#     numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns
#     df_zscore = df[numerical_cols].apply(lambda x: (x - x.mean()) / x.std())
#     zscore_threshold = 3
#     outlier_zscore = (df_zscore.abs() > zscore_threshold).any(axis=1)
#     Q1 = df[numerical_cols].quantile(0.25)
#     Q3 = df[numerical_cols].quantile(0.75)
#     IQR = Q3 - Q1
#     iqr_threshold = 1.5
#     outlier_iqr = ((df[numerical_cols] < (Q1 - iqr_threshold * IQR)) | 
#                    (df[numerical_cols] > (Q3 + iqr_threshold * IQR))).any(axis=1)
    
#     # Create a combined outlier column for the segment
#     df[f'Outlier_{segment}'] = outlier_zscore | outlier_iqr
#     return df

# # Segment the data based on the 'running' cluster labels
# data_running = data[data['running'] == 1].copy()
# data_stationary = data[data['running'] == 0].copy()

# # Detect outliers in each segment and update the original dataframe
# data.loc[data_running.index, 'Outlier_Running'] = detect_outliers(data_running, 'Running')['Outlier_Running']
# data.loc[data_stationary.index, 'Outlier_Stationary'] = detect_outliers(data_stationary, 'Stationary')['Outlier_Stationary']

# # Fill NaN values for the outlier columns with False
# data['Outlier_Running'].fillna(False, inplace=True)
# data['Outlier_Stationary'].fillna(False, inplace=True)

# # Display the head of the dataframe to verify the new columns
# data.head()

# Save to CSV

In [13]:
data_parsed = data_parsed.drop(['RPM_Mean'], axis=1)

In [14]:
data_parsed.to_csv('labeled_augumented_cleaned_ar41_for_ulb.csv', index=True)